# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **


This is a **classification** problem, because we only want to identify if the students either:

- A) Need early intervention.

    **or**


- B) Do *not* need early intervention.

It would be a **regression** problem if, for example, we had been asked to estimate:

- The number of extra hours of study needed per student to graduate.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print 'Yes:', len(student_data.loc[student_data['passed'] == 'yes'])
print 'No:', len(student_data.loc[student_data['passed'] == 'no'])
print 'All:', len(student_data)

Student data read successfully!
Yes: 265
No: 130
All: 395


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [14]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1  # minus the target column

# TODO: Calculate passing students
n_passed = len(student_data.loc[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data.loc[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=23)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:**

## Logistic Regression


**Describe one real-world application in industry where the model can be applied.**

Predicting whether a voter will vote for a Democratic or Republican candidate, based on age, income, sex, race, etc.

> Harrell, F. (2015). Regression modeling strategies: with applications to linear models, logistic and ordinal regression, and survival analysis.

**What are the strengths of the model; when does it perform well?**

- You can use multiple variations of a feature without hurting the overall model (sloppy feature engineering).

- Logistic regression will work better if there's a single decision boundary.

- Logistic regression is intrinsically simple and it has low variance.

- The output can be interpreted as a probability. This is something that comes as a nice side effect since you can use it, for example, for ranking instead of classification.

**What are the weaknesses of the model; when does it perform poorly?**

- With small training data, Logistic Regression models may overfit the data.
- Requires large sample size to achieve stable results.

**What makes this model a good candidate for the problem, given what you know about the data?**

- As a rule of thumb, in classification problems is a good idea to start with Logistic Regression to have a baseline before using "fancier" approaches.

- In the data given in this problem, the features seem to be roughly linear and the problem to be linearly separable, and Logistic Regression works well in this type of problems.


## Naive Bayes

**Describe one real-world application in industry where the model can be applied.**

Text classification, for example:

- Finding articles that are interesting for a user.
- Determining if a website discusses machine learning topics.

> Mitchell, T. M. (1997). Machine learning.

**What are the strengths of the model; when does it perform well?**

- Is very fast.
- Works well even with small training data, as the estimates are based on the joint density function.
- Performs well when you can assume that the features are conditionally independent given the class.
- When you have plenty of input features but small number of records, then Naive Bayes is a good approach.

**What are the weaknesses of the model; when does it perform poorly?**

- Assumes all the features are conditionally independent. If some of the features are dependent on each other the prediction might be poor (even though this assumption rarely holds in practice).

**What makes this model a good candidate for the problem, given what you know about the data?**

- We have plenty of features (30, or 48 after converted to dummy variables) and not too many training records (only 300 after splitting the dataset in train/test sets). Naive Bayes tends to work well with many features and small data.

## Support Vector Machines

**Describe one real-world application in industry where the model can be applied.**

Support vector machines have been successfully used in text recognition. For example, it was used in the construction of a system for license-plate recognition.

> K. K. Kim, K. I. Kim, J. B. Kim and H. J. Kim, "Learning-based approach for license plate recognition," Neural Networks for Signal Processing X, 2000. Proceedings of the 2000 IEEE Signal Processing Society Workshop, Sydney, NSW, 2000, pp. 614-623 vol.2.

**What are the strengths of the model; when does it perform well?**

- High accuracy.
- Good theoretical guarantees regarding overfitting.

**What are the weaknesses of the model; when does it perform poorly?**

- Picking/finding the right kernel can be a challenge.
- SVMs can be very inefficient to train when having many training examples.
- Very hard to use in "industry scale" applications.
- No standardized way for dealing with multi-class problems; fundamentally a binary classifier.

**What makes this model a good candidate for the problem, given what you know about the data?**

- Because SVMs work very well in binary classification problems.
- The inefficiency of training SVMs is not a problem because our training set only has 300 examples.


#### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [26]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [32]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# TODO: Initialize the three models
classifiers = [
    LogisticRegression(),
   GaussianNB(),
    SVC()
]

# TODO: Set up the training set sizes
training_sizes = [100, 200, 300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for classifier in classifiers:
    for training_size in training_sizes:
        train_predict(classifier, X_train[:training_size], y_train[:training_size], X_test, y_test)
    print '-' * 100, '\n'

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8189.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7755.

Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0019 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.8082.

Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0042 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8258.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.8267.

---------------------------------------------------------------------------------------------------- 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8387.
Made predictions 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0022 sec         |       0.0004 sec       |      0.8189      |      0.7755     |
| 200               |      0.0029 sec         |       0.0004 sec       |      0.8406      |      0.8082     |
| 300               |      0.0035 sec         |       0.0004 sec       |      0.8258      |      0.8267     |

** Classifer 2 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0019 sec         |       0.0006 sec       |      0.8387      |      0.8027     |
| 200               |      0.0008 sec         |       0.0004 sec       |      0.7940      |      0.8133     |
| 300               |      0.0010 sec         |       0.0005 sec       |      0.7960      |      0.7808     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0023 sec         |       0.0013 sec       |       0.8551     |      0.8079     |
| 200               |      0.0035 sec         |       0.0026 sec       |       0.8649     |      0.8079     |
| 300               |      0.0099 sec         |       0.0057 sec       |       0.8724     |      0.8054     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I selected a **Support Vector Machine** model because they are very accurate in binary classification problems. The fact that SVMs are slow to train is not relevant in this case because we don't have too much data, and even though the training time was 10x slower than Naive Bayes and 3x slower than the Logistic Regression model, it took only 0.0099 sec to train with the dataset of size 300.

Also, I chose the SVM model because the f1 score seemed to be more consistent with the three different training sizes: 80.79 for 100, 80.79 for 200 and 80.54 for 300.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Here is an example of how Support Vector Machines work in classifiying the students:

Let's imagine that we have a group of students, and some need intervention and some don't. The students are placed in different locations of a soccer field based on their features (let's keep it to two for simplicity): absences and study time. The students that need intervention wear a red t-shirt, and the students that don't wear a blue t-shirt. A Support Vector Machine will try to put a line in the field in such a way that all the blue t-shirts will be in one side of the line, and all the red t-shirts will be in the other side, trying to have as big a gap on either side of the line as possible.

In an ideal world there is a line that would perfectly separate all the blues in one side, and all the reds in the other side. But in practice this doesn't happen very often. Even if the SVM finds a line that will keep the majority of the t-shirts on each side, there will be a few that will be in the wrong side. When this happens SVM uses a trick.  First, SVM will use its power to levitate all the students, and then will use one of the big banners from the fans in the public and slip it between the students to separate them.

The students are the data.

The line is the classifier.

The levitation trick is called kernelling.

And the banner is the hyperplane.

Then, when a new student enters the field without a t-shirt, based on the line and the banner we can determine if it should wear a red or blue t-shirt (testing).

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [78]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {
    'kernel': ['rbf'],
    'C': [1, 50, 75, 100, 125],
    'gamma': [1e-1, 1e-2, 1e-3, 1e-4]
}

# TODO: Initialize the classifier
clf = SVC(random_state=23)
# clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=23, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8168.
Made predictions in 0.0016 seconds.
Tuned model has a testing F1 score of 0.8571.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F_1 score is:

- Training F1 score of 0.8168.

- Testing F1 score of 0.8571.

As we can see, even though the training F1 score decreased from 0.8724 to 0.8168, the testing f1 scored improved from 0.8054 to 0.8571. This suggests that before the tunning the model was overfitting to the training set and not performing that well in the testing set. But after the parameters kernel, C and gama were tunned, the model improved at generalizing from unseen data.



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.